In [8]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [29]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import random
import pickle
import cv2
import os

In [30]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model along with the input shape to be "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        # if we are using "channels first", update the input shape and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
        # CONV => RELU => POOL
        x = 32
        model.add(Conv2D(x, (3, 3), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.2))
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(x*2, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(x*2, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(x*4, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(x*4, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))
        model.add(Conv2D(x*8, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))
        
        # FC => RELU 
        model.add(Flatten())
        model.add(Dense(2048))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.45))
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        # return the constructed network architecture
        return model

In [38]:
args = {"dataset": "C:\\Users\\aakas\\CodeStorm\\dataset" } #Come back later for more 

# initialize the number of epochs to train for, initial learning rate, batch size, and image dimensions
EPOCHS = 100
INIT_LR = 3e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)
# initialize the data and labels
data = []
labels = []
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(args["dataset"])))
random.seed(21)
random.shuffle(imagePaths)

[INFO] loading images...


In [39]:
# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
    # extract the class label from the image path and update the labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [40]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(data.nbytes / (1024 * 1000.0)))
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.2, random_state=21)

[INFO] data matrix: 548.86MB


In [41]:
# constructing the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, width_shift_range=0.3,height_shift_range=0.3, shear_range=0.25, zoom_range=0.25, 
                         horizontal_flip=True, fill_mode="nearest")

In [ ]:
# initialize the model
print("[INFO] compiling model...")
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / 100)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1, min_delta=1e-4, mode='min')
# train the network
print("[INFO] training network...")
H = model.fit(
    x=aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    callbacks = [mcp_save, reduce_lr_loss],
    steps_per_epoch=len(trainX) // BS,
    epochs=100, verbose=1)

[INFO] compiling model...
[INFO] training network...
Epoch 1/100
63/63 [==============================] - 69s 1s/step - loss: 3.4459 - accuracy: 0.2035 - val_loss: 4.6384 - val_accuracy: 0.1198 - lr: 0.0030
Epoch 2/100
63/63 [==============================] - 63s 995ms/step - loss: 3.1126 - accuracy: 0.2365 - val_loss: 3.1970 - val_accuracy: 0.2083 - lr: 0.0030
Epoch 3/100
63/63 [==============================] - 61s 976ms/step - loss: 2.9555 - accuracy: 0.2703 - val_loss: 2.9591 - val_accuracy: 0.2200 - lr: 0.0030
Epoch 4/100
63/63 [==============================] - 61s 965ms/step - loss: 2.8017 - accuracy: 0.2700 - val_loss: 2.0617 - val_accuracy: 0.3163 - lr: 0.0030
Epoch 5/100
63/63 [==============================] - 63s 1s/step - loss: 2.6496 - accuracy: 0.2940 - val_loss: 2.4005 - val_accuracy: 0.2711 - lr: 0.0030
Epoch 6/100
63/63 [==============================] - 62s 992ms/step - loss: 2.5166 - accuracy: 0.2965 - val_loss: 2.1392 - val_accuracy: 0.3163 - lr: 0.0030
Epoch 7/100

In [37]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = 100
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig("plot1.png")